In [1]:
import os
%pwd

'h:\\Projects\\TextSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'h:\\Projects\\TextSummarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int    

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

In [5]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

h:\Tampere\Softwares\conda\envs\txtenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-20 15:55:44,452] 54 datasets - INFO - PyTorch version 2.5.1 available.


In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load the dataset
        samsum_data_pt = load_from_disk(self.config.data_path)

        # Define the training arguments
        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_dir=self.config.root_dir,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            report_to="none"
        )

        trainer = Trainer(
            model=model_pegasus, processing_class=tokenizer,
            data_collator=seq2seq_data_collator,
            args=training_args,
            train_dataset=samsum_data_pt['test'],    # Use the test dataset for training as the dataset is small and easy to train. Change to 'train' for larger datasets.
            eval_dataset=samsum_data_pt['validation']
        )

        # Train the model
        trainer.train()

        # Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_model"))

        # Save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_tokenizer"))
    

In [7]:
!pip3 install --upgrade accelerate
!pip3 uninstall -y transformers accelerate
!pip3 install transformers accelerate

Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Successfully uninstalled transformers-4.46.2
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   --------- ------------------------------ 2.4/10.0 MB 12.2 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.0 MB 12.8 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.0 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 8.0 MB/s eta 0:00:00
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)


In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-11-20 16:00:06,399] 16 TextSummarizerLogger - INFO - yaml file config\config.yaml read successfully.
[2024-11-20 16:00:06,421] 16 TextSummarizerLogger - INFO - yaml file params.yaml read successfully.
[2024-11-20 16:00:06,434] 35 TextSummarizerLogger - INFO - Directory artifacts already exists.
[2024-11-20 16:00:06,437] 35 TextSummarizerLogger - INFO - Directory artifacts/model_trainer already exists.


: 